# Important: This notebook processes the data with 150nts only!

In [1]:
cd trimmed-150nts/

/Users/yoshikivazquezbaeza/Documents/PDF/KnightLaboratory/HastyWater/trimmed-150nts


In [2]:
%matplotlib inline
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, qiime2 as q2

In [3]:
from biom import load_table, Table
from biom.util import biom_open
from skbio import TreeNode

from qiime2.plugins import diversity, feature_table, metadata, taxa, emperor, diversity

def load_mf(fn, index='#SampleID'):
    _df = pd.read_csv(fn, sep='\t', dtype=str, keep_default_na=False, na_values=[])
    _df.set_index(index, inplace=True)
    return _df

The tables that we'll deal with are the ones normalized for acinetobacter, see the previous notebooks for details about this.

The phylogenetic tree and taxonomic assignments are generated from the representative sequences. Note those processing steps are not shown here as they were executed in our local compute cluster. They can be found in `/home/yovazquezbaeza/research/hasty-water/trimmed-150nts/`.

The commands executed were (roughly):

```bash
qiime deblur denoise-16S --i-demultiplexed-seqs single-end-demux.filtered.qza --p-trim-length 150 --o-representative-sequences rep-seqs-deblur.qza --o-table table-deblur.qza --o-stats deblur-stats.qza

# assign taxonomy greengenes
qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza --i-reads rep-sequences.qza --o-classification taxonomy.greengenes.qza --p-n-jobs 4

qiime metadata tabulate --m-input-file taxonomy.greengenes.qza --o-visualization taxonomy.greengenes.qzv
```

I also created a fragment-inserted phylogenetic tree based on SEPP and these fragments (?????):

```bash
qiime fragment-insertion sepp-16s-greengenes --i-representative-sequences rep-seqs-deblur.qza --o-tree sepp-tree.rooted.qza --o-placements sepp-placements.qza --p-threads 31
```

# Alpha diversity

In [4]:
table = q2.Artifact.load('feature-table.even.18813.normalized.qza')
tree = q2.Artifact.load('insertion-tree.qza')

In [5]:
diversity.methods.alpha_phylogenetic(table, tree, 'faith_pd').alpha_diversity.save('faith_pd.qza')

for metric in ['observed_otus', 'shannon']:
    diversity.methods\
    .alpha(table, metric)\
    .alpha_diversity\
    .save('%s.qza' % metric)

Add the alpha diversity information to the mapping file.

In [6]:
mf = q2.Metadata.load('../mapping-file.tsv').to_dataframe()

In [7]:
for metric in ['observed_otus', 'shannon', 'faith_pd']:
    mf[metric] = q2.Artifact.load(metric + '.qza').view(pd.Series)

mf.to_csv('mapping-file.alpha.tsv', sep='\t')

Create taxonomic bar charts

In [8]:
taxonomy = q2.Artifact.load('taxonomy.qza')

In [9]:
taxa.visualizers.barplot(table,
                         taxonomy,
                         q2.Metadata(mf)).visualization.save('taxonomy.barplot.qzv')

'taxonomy.barplot.qzv'

Beta diversity

In [10]:
diversity.methods.beta_phylogenetic_alt(table=table,
                                        phylogeny=tree,
                                        metric='unweighted_unifrac',
                                        n_jobs=2)\
.distance_matrix.save('unweighted.unifrac.dm.qza')

'unweighted.unifrac.dm.qza'

In [11]:
diversity.methods.beta_phylogenetic_alt(table=table,
                                        phylogeny=tree,
                                        metric='weighted_unifrac',
                                        n_jobs=2)\
.distance_matrix.save('weighted.unifrac.dm.qza')

'weighted.unifrac.dm.qza'

In [12]:
diversity.methods.pcoa(q2.Artifact.load('unweighted.unifrac.dm.qza')).pcoa.save('unweighted.unifrac.pcoa.qza')
diversity.methods.pcoa(q2.Artifact.load('weighted.unifrac.dm.qza')).pcoa.save('weighted.unifrac.pcoa.qza')

/Users/yoshikivazquezbaeza/git_sw/scikit-bio/skbio/stats/ordination/_principal_coordinate_analysis.py:114: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.02957537690504521 and the largest is 5.5273874406739525.
  RuntimeWarning


'weighted.unifrac.pcoa.qza'

In [13]:
emperor.visualizers.plot(q2.Artifact.load('unweighted.unifrac.pcoa.qza'),
                         q2.Metadata.load('mapping-file.alpha.tsv')).visualization.save('unweighted.unifrac.plot.qzv')

emperor.visualizers.plot(q2.Artifact.load('weighted.unifrac.pcoa.qza'),
                         q2.Metadata.load('mapping-file.alpha.tsv')).visualization.save('weighted.unifrac.plot.qzv')

'weighted.unifrac.plot.qzv'